In [2]:
#Habilitar intellisense
%config IPCompleter.greedy=True

In [3]:
import pandas as pd
import numpy as np
import nltk
from xgboost.sklearn import XGBClassifier
import string
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import re
import pickle
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier


In [5]:
data = pd.read_csv('imdb_dataset.csv',encoding="ISO-8859-1")
data.head()

,Review,Label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [12]:
# print(data.shape) # (50000, 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  50000 non-null  object
 1   Label   50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [14]:
data_prueba=data.sample(frac=0.2)
palabrasParada = set( nltk.corpus.stopwords.words('english') + list(string.punctuation)+["...","..","hr"])

In [15]:
# data_prueba

,Review,Label
24491,I avoided this film as a boy because I thought...,pos
33530,This makes the third Errol Morris movie I've s...,neg
38010,"Lillian Hellman, one of America's most famous ...",pos
35321,I have read the novel Reaper of Ben Mezrich a ...,neg
36407,I caught this movie on Sci-Fi before heading i...,neg
...,...,...
19629,My Wife and I saw this movie once in 1989 and ...,pos
21154,"In the early '80s, I recorded Honky Tonk Freew...",pos
5669,I am almost tempted to demand my money back fr...,neg
6646,"Holy crap, the beginning picked up where the f...",neg


In [16]:
#Guardando data -> 20% = 10.000
# # #Crear DataFrame de 10.000 
# df = pd.DataFrame(poblacion) 
# muestra = df.sample(n = 10000)
# muestra.head() #Es aleatorio, cada vez hay nuevos valores -> Resultados deberian ser parecidos 

#Exportar DataFrame
data_prueba.to_csv ('imdb_dataset_prueba.csv')

In [22]:
def sinhtml(conHTML):
    limpiar_html = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    sin_html = re.sub(limpiar_html, '', conHTML)
    return sin_html

def quitar_html(palabras):
    array = []
    for palabra in  palabras:
        sinHtml = sinhtml(palabra)
        array.append(sinHtml)
    return array

def quitarComillas(palabras):
    array = []
    for palabra in  palabras:
        palabra = palabra.strip("'")
        palabra = palabra.strip("`")
        array.append(palabra)
    return array

def Tokenizar(texto):
    texto=texto.lower()
    palabras = nltk.word_tokenize(texto)#separa las palabras
    return [palabra for palabra in palabras if palabra not in palabrasParada]#quita stopwords
def listToString(lista):  
    str1 = " " 
    return (str1.join(lista)) 


In [23]:
listaDeComentarios=[]
for index, row in data_prueba.iterrows():
    tokenizado=Tokenizar(row['Review'])
    sinHTML=quitar_html(tokenizado)
    sinComillas=quitarComillas(sinHTML)
    lista=listToString(sinComillas)  
    listaDeComentarios.append(lista)
listaDeComentarios[0]

'various nudity scenes reviewers referred poorly done body double obviously used ms. pacula reluctant scenes perhaps turned role offer. br br otherwise movie worse typical canadian movies reviewers pointed canadian movies generally poorly written lack entertainment value movies watchers hoping get perhaps canadian movie producers consciously trying  de-commercialize  movies forgotten important thing movies definition commercial thing ....'

In [24]:
def get_problem_vocabulary(normalized_corpus):
    all_tokens = [] 
    for document in normalized_corpus:
        all_tokens.extend(document.split())  
    all_tokens_sorted = sorted(set(all_tokens))
    
    token_and_position = {}
    for i, token in enumerate(all_tokens_sorted):
        token_and_position[token] = i
    
    return token_and_position
problem_vocabulary = get_problem_vocabulary(listaDeComentarios)
print("Tamaño del lista: ",len(problem_vocabulary))


Tamaño del lista:  69440


In [25]:
def one_hot_vector(document, problem_vocabulary):
    vector = np.zeros(len(problem_vocabulary),dtype=int)
    for token in document.split():
        vector[problem_vocabulary[token]] = 1
    return vector
def one_hot_matriz(lista):
    matriz=[]
    for i in lista:
        matriz.append(one_hot_vector(i, problem_vocabulary))
    return matriz
        
matriz=one_hot_matriz(listaDeComentarios)

In [26]:
print("Numero de comentarios utilizados: " ,len(data_prueba))

Numero de comentarios utilizados:  10000


In [27]:
matrizDic=[]
i=0
for index, row in data_prueba.iterrows():
    z = {'tokenizado':matriz[i],'clasificacion':row['Label']}
    i=i+1
    matrizDic.append(z)


In [28]:
listaParaEntrenar=[]
for i in matrizDic:
    if(i["clasificacion"]=="pos"):
        listaParaEntrenar.append([i["tokenizado"], 1])
    else:
        listaParaEntrenar.append([i["tokenizado"], 0])

In [29]:
df = pd.DataFrame(listaParaEntrenar)
dfExpandido = df[0].apply(pd.Series)
dfExpandido["res"]=df[1]
tamMat=len(problem_vocabulary)

In [30]:
X = dfExpandido.iloc[:, np.r_[0:tamMat]]
y = dfExpandido["res"]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training, 30% test
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7000, 69440) (3000, 69440) (7000,) (3000,)


In [32]:
treeV4=RandomForestClassifier(n_estimators=88)

In [33]:
treeV4.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=88,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
print('accuracy del clasificador - version 4 : {0:.2f}'.format(accuracy_score(y_test, treeV4.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 4: \n {0}'.format(confusion_matrix(y_test, treeV4.predict(X_test))))
# precision 
print('precision del clasificador - version 4 : {0:.2f}'.format(precision_score(y_test, treeV4.predict(X_test))))
# recall
print('recall del clasificador - version 4 : {0:.2f}'.format(recall_score(y_test, treeV4.predict(X_test))))
# f1
print('f1 del clasificador - version 4 : {0:.2f}'.format(f1_score(y_test, treeV4.predict(X_test))))

accuracy del clasificador - version 4 : 0.83
matriz de confusión del clasificador - version 4: 
 [[1296  214]
 [ 283 1207]]
precision del clasificador - version 4 : 0.85
recall del clasificador - version 4 : 0.81
f1 del clasificador - version 4 : 0.83


In [35]:
ruta_archivo_clasificador = os.path.join("Clasificador", "tree_v4.pkl")
archivo_clasificador = open(ruta_archivo_clasificador, "wb")
pickle.dump(treeV4, archivo_clasificador)
archivo_clasificador.close()

In [36]:
ruta_archivo_vocabulario = os.path.join("Clasificador","VocabularioProblema.pkl")

#Abrir el archivo para escribir contenido binario
archivo_vocabulario = open(ruta_archivo_vocabulario, "wb")

pickle.dump(problem_vocabulary, archivo_vocabulario,protocol=2)

archivo_vocabulario.close()